## Step 0: Importing modules

In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import yfinance as yf

## Step 1: Getting stock price data as a dataframe

In [ ]:
stock_ticker = "MSFT"  # Microsoft
data = yf.download(stock_ticker, start="2023-01-01", end="2024-01-01")  # Get last year's daily price data for MSFT

In [ ]:
type(data)  # yfinance (Yahoo Finance) automatically returns a Pandas DataFrame

## Step 2: Getting returns of stock data with Pandas

In [ ]:
data.head(5)  # Inspect the first 5 rows of the dataframe

The **return** of a stock is defined as the assets's change in value over some period of time. In quantitative investing and algorithmic trading, we often use the **percent return** of an asset over some period (minute by minute, day over day, etc.) in our modeling rather than the exact price over time.

This is because while the scale of an asset's price can change greatly over time (e.g. BTC started at 13 in 2012, and is now near 100,000), returns represented as a percentage are *scale free* and *stationary*, making statistical figures like mean, variance, correlations, etc. more useful. The use of percent returns will become more apparent in the following lectures!

In [ ]:
data["Pct Return"] = data['Close'].pct_change()  # Calculate daily percent return
data["Pct Return"]

## Step 3: Analyzing Summary Statistics

Now that we have daily pct returns data, we can looks at some quantitative metrics of the performance of our asset over the time frame of our data. For example, mean, standard deviation, highs, lows, and percentiles all give us a better picture of the performance of our asset (**Week 2 foreshadowing**: particularly mean and stardard deviation!)

In [ ]:
data["Pct Return"].describe()  # .describe() gives us several useful summary stats for a numerical column

Here we see that the average (mean) daily return (percent change) of MSFT in 2023 is 0.1936%, and see other useful metrics.

We can also use specific methods for various summary statistics:

In [ ]:
#data['Pct Return'].mean()  # Mean
#data['Pct Return'].median()  # Median
#data['Pct Return'].mode()  # Mode
#data['Pct Return'].std()  # Standard Deviation
#data['Pct Return'].var()  # Variance
#data['Pct Return'].sum()  # Sum
#data['Pct Return'].count()  # Count (number of rows)
#data['Pct Return'].min()  # Minimum value
#data['Pct Return'].max()  # Maximum Value

## Step 4: Basic Feature Engineering for Algorithmic Trading Models

In order to gain some fluency with Pandas, we will create some features.

"Feature engineering is the process of selecting, manipulating and transforming raw data into features that can be used in supervised learning. It's also necessary to design and train new machine learning features so it can tackle new tasks. A “feature” is any measurable input that can be used in a predictive model."

Let's start with a *Simple Moving Average*. This is defined as the average of the current element at a time and the last n-1 elements before it. SMAs are used widely in various trading strategies (such as moving average crossover) and technical indicators (such as Bollinger Bands). More on that next week!

In [ ]:
data["SMA"] = data["Close"].rolling(5).mean()  # SMA of last 5 days of MSFT close price
data["SMA"]

Another useful feature used widely in algorithmic trading is *z-scores*. 

"In algorithmic trading, a "z-score" is a statistical measure that indicates how far away a particular data point (like a stock price) is from the historical average price, expressed in terms of standard deviations, essentially showing whether the current price is considered "normal" or an outlier compared to past price movements; traders use z-scores to identify potential trading opportunities based on significant deviations from the mean, allowing them to potentially capitalize on price reversals or strong trends."

**Note**: In an algorithm that is actually trading live, it would be unrealistic to use the mean and std of the entire time series we trade on, as we cannot see the future, nor do we want calculations to be too intense with all emassed previous data. Hence, z-scores tend to be calculated on a rolling basis in trading algorithms.

In [ ]:
rolling_mean = data["Close"].rolling(30).mean()  # This is the 30-day SMA of close
rolling_std = data["Close"].rolling(30).std()  # 30 day rolling STD of close
data["Z-score"] = ((data["Close"] - rolling_mean) / rolling_std)  # 30-day rolling zscore of close
data["Z-score"]

## Step 5: Plotting Price and Returns

It's hard to interpret all this useful data without being able to visualize it. To do so, we can make some basic plots with matplotlib. Matplotlib has many different functionalities for building various plots, which can be found in the docs here: https://matplotlib.org/3.5.3/api/_as_gen/matplotlib.pyplot.html

Pro tip: ChatGPT can be a good time-saver for getting code for specific visualization (just take the code with a grain of salt)!

For now, let's plot the daily price and returns of MSFT in 2023:

In [ ]:
# Plotting Daily Close Price

plt.figure(figsize=(10, 6))
plt.plot(data.index, data['Close'], label="MSFT Close Price")
plt.title("MSFT Daily Close Price 2023")
plt.xlabel("Date")
plt.ylabel("Close Price (USD)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Plotting Daily Returns:

plt.figure(figsize=(10, 6))
plt.plot(data.index, data['Pct Return'], label="MSFT Pct Change")
plt.title("MSFT Daily Pct Return 2023")
plt.xlabel("Date")
plt.ylabel("MSFT Pct Change")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

These plots also highlight the scaling and stationarity advantages of using returns rather than just price.

**For fun**: Try plotting the SMA and closing price together!

## Step 6: Using Lagging Regression to Predict Future Prices

Regression is both one of the simplest and commonly used statistical model in quantitative finance. By using the previous (lagging) values of variables as our feature vectors (inputs) and the current percent return as our label vector (target), we can learn a liner regression model that takes the last values of the feature vector and predicts the next percent return. Such a model, if accurate, is powerful because it can inform our algorithm's decision to buy/hold or sell when the next return is estimated to be positive or negative respectively (simplified).

The following are just some examples of lagging features we could use in such a model (possibly in combination):
1. Lagging "order book" data such as lagging close prices, percentage change, high/low/volume
2. Lagging rolling window features, such as previous SMA values or z-scores
3. More complex features such as Technical Indicators (more on that in following weeks)
4. Sector/market prices, such as the lagging value of the S&P 500 (SPY)
5. Lagging stock data such as dividend-price ratio, earnings-price ratio, etc.
And more.

For the sake of simplicity, let's use the last 3 day's returns for MSFT to try and predict it's future returns.

### Step 6a: Preparing Training and Testing Data

In [ ]:
data['Lag1'] = data['Pct Return'].shift(1)  # 1-day lag
data['Lag2'] = data['Pct Return'].shift(2)  # 2-day lag
data['Lag3'] = data['Pct Return'].shift(3)  # 3-day lag

# Drop SMA and Z-score columns since they are missing first 30 values
data.drop(["SMA", "Z-score"], axis=1, inplace=True)  

data.dropna(inplace=True)  # Since the first three days won't have values for all lags, drop them

In [ ]:
X = data[['Lag1', 'Lag2', 'Lag3']]  # Feature Vector
y = data['Pct Return']  # Labels

Since we want to be able to analyze the performance of our model on unseen data, let's keep the first 80% of our data as training data and evaluate it on the unseen testing data.

In [ ]:
test_size = 0.2  # 20% of data for testing
split_index = int(len(X) * (1 - test_size))  # Calculate the split index

X = sm.add_constant(X)  # Adding constant for the intercept term

X_train = X.iloc[:split_index]
X_test = X.iloc[split_index:]
y_train = y.iloc[:split_index]
y_test = y.iloc[split_index:]

In [ ]:
print(X_train.shape)  # 196 days of training data
print(X_test.shape)  # 50 days of test data

**Question to consider**: Would it be smart to randomly select 80% of the days as training data and the rest as test data, or would we want to keep the training data and test data as continuous time intervals like we did above? Why or why not?

In [ ]:
data[["Pct Return", "Lag1", "Lag2", "Lag3"]].head(4)  # Confirming our data looks good

### Step 6b: Learning our Linear Regression Model

For this, we will use Ordinary Least Squares (OLS) from the Statsmodels package

In [ ]:
ols = sm.OLS(y_train, X_train).fit()  # Fit our OLS model to our data

Now we can see the constant (where the regression "line" crosses the y-axis) and the coefficients for each parameter that the OLS regression fit to.

Given new feature values (such as lag1, lag2, lag3), the value predicted by our model will be equal to the sum of the features multiplied by their coefficients plus the constant.

In [ ]:
ols.params

We can also see the R^2 value of our model, which measures the % of the target variable's variability the model can explain. If R^2 = 1, the model is perfectly fit and explains all variability in the target variable. If R^2 = 0, the model explains none of the variability of the target variable, and may as well just always predict the mean value of the target.

In [ ]:
ols.rsquared

**Question to consider**: How well do you think the model explains the future returns' variability? Are you suprised by this value? Why or why not? (Hint: Would you expect financial data to be "noisy"?)

### Step 6c: Testing our Model on Unseen Data + Analysis

Now let's try testing our model on the unseen testing data, and comparing it's predictions to the true values.

In [ ]:
y_pred = ols.predict(X_test)  # Predict the testing pct_change using the test values

We can plot the predicted and real values and compare visually:


In [ ]:
test_dates = data.reset_index()["Date"].iloc[split_index:]  # Days we are testing on

plt.figure(figsize=(10, 6))
plt.plot(test_dates, y_test, label="Real Pct Change")
plt.plot(test_dates, y_pred, label="Predicted Pct Change")
plt.title("Real Vs. Predicted MSFT Daily Pct Return")
plt.xlabel("Date")
plt.ylabel("Pct Change")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

To quantitatively evaluate our model, we can calculate the **mean squared error**, which uses the following formula:

In [ ]:
mse = ((y_test - y_pred) ** 2).mean()
print(mse)

**Question to consider**: Do you think this MSE is high or low, good or bad? Consider the explicit formula, the scale of pct_change, and other factors.

## Step 7: Try Stuff Yourself

**For less experienced**: Now that you know how to get stock price data, work with data in Pandas, make plots, and run regression, try making your own regression model! See if you can find good lagging features to predict whichever asset's returns you pick. Feel free to refer to documentation and ChatGPT!

**For more experienced**: Do the above, and then backtest your regression model by turning your rolling regression predictions into buy/sell/hold values (can use a for loop or pandas.DataFrame.cumprod).